# Voiceshell Work Notebook

*Adapted from Allison Parrish's Understanding Word Vectors notebook*

#### Python Package Installation for Speech Recognition

*Run in terminal to install SpeechRecognition.py & PocketSphinx engine:*

`pip install SpeechRecognition`

`pip install pocketsphinx`

#### Importing required libraries:

In [2]:
# IMPORTS
import math
import random
from __future__ import unicode_literals
import spacy
import speech_recognition as sr
import os
import csv
import numpy as np
from numpy import dot
from numpy.linalg import norm

#### Defining basic mathematical functions for vector math:

In [3]:
def meanv(coords):
    sumv = [0] * len(coords[0])
    for item in coords:
        for i in range(len(item)):
            sumv[i] += item[i]
    mean = [0] * len(sumv)
    for i in range(len(sumv)):
        mean[i] = float(sumv[i]) / len(coords)
    return mean

In [4]:
def cosine(v1, v2):
    if norm(v1) > 0 and norm(v2) > 0:
        return dot(v1, v2) / (norm(v1) * norm(v2))
    else:
        return 0.0

#### Defining Spacy/Word Vector low level functions:

In [5]:
def sentvec(s):
    sent = nlp(s)
    word_vectors = [w.vector for w in sent]
    return meanv(word_vectors)

In [6]:
def spacy_closest_sent(space, input_str, n=1):
    input_vec = sentvec(input_str)
    return sorted(space,
                  key=lambda x: cosine(np.mean([w.vector for w in x], axis=0), input_vec),
                  reverse=True)[:n]

#### Creating Spacy processor and loading a corpus:

In [7]:
nlp = spacy.load('en')
# doc = nlp(open("corpus/andrews1.txt").read().replace('\n', ' ').replace('  ', ' '))

In [8]:
with open('revisedCorpus/voiceshell_audio_LUT.csv', 'r') as f:
    reader = csv.reader(f)
    all_lines = list(reader)
    
sentences = []
for line in all_lines:
    sentence = line[0]
    if not sentence.isspace():
        sentences.append(nlp(sentence))

#### Splitting corpus into individual, complete sentences:

In [92]:
# spacySentences = list(doc.sents)

#### Define list of prompts to user:

In [9]:
inquiries = ["How are you feeling?", "Tell me a quote you like.", "What are the first three words that pop into your head?", "What day is it today?", "What's your favourite colour?", "Where are you right now?", "What's your job?", "What do you like to eat?", "What animal would you want as a pet?"]

In [10]:
print(sentences[22])

#### Defining user input prompts, and getting input to process, return result:

In [11]:
inquiry = random.choice(inquiries)
print(inquiry)
user_input = input()

for sent in spacy_closest_sent(sentences, user_input):
    sentNum = sentences.index(sent)
    print("---")
    print(sentNum)
    print(sent.text)
    print("---")

What's your job?
designer
---
2
A buckling dent.
---


/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


#### Setting up basic SpeechRecognition input:
[Source](http://www.codesofinterest.com/2017/03/python-speech-recognition-pocketsphinx.html)

In [14]:
# get microphone input
r = sr.Recognizer()
with sr.Microphone() as source:
    print("Just a moment, calibrating the microphone...")
    # 5 second listen and find ambient noise level
    r.adjust_for_ambient_noise(source, duration=5)
    inquiry = random.choice(inquiries)
    
    # read question aloud and print it
    os.system("say '" + inquiry + "'")
    print(inquiry," (Speak aloud, please.)")
    audio = r.listen(source)
    
# recognise speech with Sphinx
try:
    recogSpeech = r.recognize_sphinx(audio) + "'"
    print("I heard you say: ", recogSpeech)
    for sent in spacy_closest_sent(sentences, recogSpeech):
        print("----------")
        print("...")
        print(sent.text)
        print("----------")
except sr.UnknownValueError:
    print("Sorry, I couldn't understand you.")
except sr.RequestError as e:
    print("Sphinx error; {0}".format(e))

Just a moment, calibrating the microphone...
How are you feeling?  (Speak aloud, please.)
I heard you say:  it's just that'
----------
...
I was thinking of you. How you will find out.
----------


/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [61]:
os.system("say 'White Saris, by Angela Andrews'")

0

#### Testing lookup of audio file

In [13]:
with open('revisedCorpus/voiceshell_audio_LUT.csv', 'r') as f:
    reader = csv.reader(f)
    all_lines = list(reader)
    
audio_lookup_table ={}
for line in all_lines:
    sentence = line[0]
    filename = line[1]
    audio_lookup_table[sentence] = filename
    
example_sentence = all_lines[0][0]
print("First sentence is {}".format(example_sentence))
example_audiofile = audio_lookup_table[example_sentence]
print("Look for that sentence in the file {}".format(example_audiofile))

First sentence is White Saris
Look for that sentence in the file PoemsAudio/Andrews/andrews0.wav


In [108]:
sample2 = "I carried the lamb in a sack on my horse the tongue hanging grey and limp. "
example_audiofile = audio_lookup_table[sample2]
print("Look for that sentence in the file {}".format(example_audiofile))

Look for that sentence in the file PoemsAudio/Smith/smith1.wav


#### Playing with Python WAV Audio playback

In [109]:
example_audiofile = audio_lookup_table[sent.text]
print("Look for that sentence in the file {}".format(example_audiofile))

Look for that sentence in the file PoemsAudio/Smith/smith4.wav


In [111]:
import wave, sys, pyaudio

# inquiry = random.choice(inquiries)
# print(inquiry)
# user_input = input()

# for sent in spacy_closest_sent(sentences, user_input):
#     sentNum = sentences.index(sent)
#     print("---")
#     print(sentNum)
#     print(sent.text)
#     print("---")

#     audioFile = 'PoemsAudio/Andrews/andrews{}.wav'.format(sentNum)

# sound = wave.open(audioFile)

print(example_audiofile)

sound = wave.open(example_audiofile)
p = pyaudio.PyAudio()
chunk = 1024
stream = p.open(format = p.get_format_from_width(sound.getsampwidth()), channels = sound.getnchannels(), rate = sound.getframerate(), output = True)
data = sound.readframes(chunk)
while len(data) > 0:
    stream.write(data)
    data = sound.readframes(chunk)
stream.stop_stream()
stream.close()

p.terminate()

PoemsAudio/Smith/smith4.wav


In [102]:
print(sent.text)

Small bubbles whirled, it rumpled where I was looking the water shadowed half-blue-blackdeep just there with duckweed floating out the yards behind all noise, the cattle swirling up air swelled with dust and bellowing. 


In [60]:
lookup_table = {}
# for each line in csv:
   1) text = column[1]
   2) filename = column[2]
   lookup_table[text] = filename

In [ ]:
audio_file = lokup_table[closest_sentence]